In [1]:
from extract import SocialETL, Count, UserETL
import hashtags as h
import pandas as pd
import warnings 
warnings.filterwarnings('always')   #suppresses all warnings
warnings.filterwarnings('ignore')

C:\Users\david\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1 parte: categorizzazione hashtags per prorus, proukr, pax

## 2 parte: scaricare df e distinzione 4 cat (no care) dalle altre

In [2]:
#scarcare df#24-02 al 24-03
m = SocialETL(query="(#eurovision OR #covid OR #pridemonth OR #grammys OR #justiceforjohnnydepp OR #oscars OR #elonmusk OR #putin OR #russia OR #ukraine OR #war)  lang:en", pages=1, recent=False)#strangerthings4 OR onepiece

Reading secret from C:\Users\david\Documents\GitHub\ds-network-analysis\data\my_secrets.yaml…

In [3]:
m.df

,id,conversation_id,referenced_tweets.replied_to.id,referenced_tweets.retweeted.id,referenced_tweets.quoted.id,author_id,in_reply_to_user_id,retweeted_user_id,quoted_user_id,created_at,...,geo.geo.bbox,geo.geo.type,geo.id,geo.name,geo.place_id,geo.place_type,__twarc.retrieved_at,__twarc.url,__twarc.version,
0,1545404597023432704,1545404597023432704,NaN,1545369454330126336,NaN,1357718765941710848,NaN,1219232377605644289,NaN,2022-07-08T13:49:02.000Z,...,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-08T13:49:15+00:00,https://api.twitter.com/2/tweets/search/all?ex...,2.11.0,NaN
1,1545404594611707908,1545404594611707908,NaN,1545404572780400640,NaN,1151202637582348294,NaN,251637411,NaN,2022-07-08T13:49:02.000Z,...,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-08T13:49:15+00:00,https://api.twitter.com/2/tweets/search/all?ex...,2.11.0,NaN
2,1545404588366278657,1545404588366278657,NaN,1545369454330126336,NaN,1466835011454189571,NaN,1219232377605644289,NaN,2022-07-08T13:49:00.000Z,...,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-08T13:49:15+00:00,https://api.twitter.com/2/tweets/search/all?ex...,2.11.0,NaN
3,1545404588223877123,1545404588223877123,NaN,1539792654820552704,NaN,4813333392,NaN,81386285,NaN,2022-07-08T13:49:00.000Z,...,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-08T13:49:15+00:00,https://api.twitter.com/2/tweets/search/all?ex...,2.11.0,NaN
4,1545404587695382529,1545404587695382529,NaN,1545385132751814666,NaN,2471461255,NaN,121514648,NaN,2022-07-08T13:49:00.000Z,...,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-08T13:49:15+00:00,https://api.twitter.com/2/tweets/search/all?ex...,2.11.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1545404380148547585,1545404380148547585,NaN,1545334544437493760,NaN,1056303825995337730,NaN,1496161188077621251,NaN,2022-07-08T13:48:11.000Z,...,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-08T13:49:15+00:00,https://api.twitter.com/2/tweets/search/all?ex...,2.11.0,NaN
96,1545404379825586179,1545404379825586179,NaN,NaN,NaN,1512362469389447180,NaN,NaN,NaN,2022-07-08T13:48:11.000Z,...,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-08T13:49:15+00:00,https://api.twitter.com/2/tweets/search/all?ex...,2.11.0,NaN
97,1545404378131185669,1545404378131185669,NaN,1545387894952583169,NaN,3245630026,NaN,23372297,NaN,2022-07-08T13:48:10.000Z,...,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-08T13:49:15+00:00,https://api.twitter.com/2/tweets/search/all?ex...,2.11.0,NaN
98,1545404377984270336,1545404377984270336,NaN,NaN,NaN,4186147403,NaN,NaN,NaN,2022-07-08T13:48:10.000Z,...,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-08T13:49:15+00:00,https://api.twitter.com/2/tweets/search/all?ex...,2.11.0,NaN


In [4]:
user_id=m.df['author_id']

print(user_id)

0     1357718765941710848
1     1151202637582348294
2     1466835011454189571
3              4813333392
4              2471461255
             ...         
95    1056303825995337730
96    1512362469389447180
97             3245630026
98             4186147403
99    1151202637582348294
Name: author_id, Length: 100, dtype: object


In [5]:
#estarre pure retweeted_user_id per edge
hashtag_for_user={}
retweet_for_user={}
verified_for_user={}
location_for_user={}
name_for_user={}
username_for_user={}

for e in user_id:
    u=UserETL(e,pages=1)                                                       #aggiungere più pagine (1pag=20)
    #preprocessing hashtag
    u.df=u.df.dropna(subset=['entities.hashtags'])                             #drop nan values on column hashtags
    u.df.reset_index()
    u.df['entities.hashtags']=u.df['entities.hashtags'].apply(eval)            #convert into dict
    u.df["entities.hashtags"] = u.df["entities.hashtags"].map(h.extract_tags)  #extract tags
    hashtag_for_user[e]=u.df["entities.hashtags"].tolist()                     #convert pandas series into a list
    #preprocessing retweet
    u.df=u.df.dropna(subset=['retweeted_user_id'])
    retweet_for_user[e]=u.df['retweeted_user_id'].tolist()
    #saving other attributes (to assigne it to a nodes as property)
    verified_for_user[e]=u.df['author.verified']
    location_for_user[e]=u.df['author.location'].tolist()
    name_for_user[e]=u.df['author.name']
   # name_for_user[e]=u.df['author.username']
    
    
    

Reading secret from C:\Users\david\Documents\GitHub\ds-network-analysis\data\my_secrets.yaml…

Reading secret from C:\Users\david\Documents\GitHub\ds-network-analysis\data\my_secrets.yaml…

Reading secret from C:\Users\david\Documents\GitHub\ds-network-analysis\data\my_secrets.yaml…

Reading secret from C:\Users\david\Documents\GitHub\ds-network-analysis\data\my_secrets.yaml…

Reading secret from C:\Users\david\Documents\GitHub\ds-network-analysis\data\my_secrets.yaml…

...

Reading secret from C:\Users\david\Documents\GitHub\ds-network-analysis\data\my_secrets.yaml…

Reading secret from C:\Users\david\Documents\GitHub\ds-network-analysis\data\my_secrets.yaml…

Reading secret from C:\Users\david\Documents\GitHub\ds-network-analysis\data\my_secrets.yaml…

Reading secret from C:\Users\david\Documents\GitHub\ds-network-analysis\data\my_secrets.yaml…

Reading secret from C:\Users\david\Documents\GitHub\ds-network-analysis\data\my_secrets.yaml…

In [15]:
user_id=[]
name=[]
hashtag=[]
retweet=[]
location=[]
verified=[]
for u,h in hashtag_for_user.items():
    user_id.append(u)
    hashtag.append(h)
for u,r in retweet_for_user.items():
    retweet.append(r)
for u,n in name_for_user.items():
    name.append(n)
for u,l in location_for_user.items():
    location.append(l)
for u,v in verified_for_user.items():
    verified.append(v)

In [20]:
print(len(verified))

77


In [28]:
#create a pandas dataframe
df_nodes=pd.DataFrame({"user_id":user_id,
                      'name': name,
                      'retweet': retweet,
                      "hashtag":hashtag,
                     'location':location,
                     'verified':verified})
df_nodes

,user_id,name,retweet,hashtag,location,verified
0,1357718765941710848,"0 PeterJohnKing4 Name: author.username, dty...",[1219232377605644289],"[[ukraine, russia]]",[Somewhere],"0 False Name: author.verified, dtype: bool"
1,1151202637582348294,0 hephaistos_ai 1 hephaistos_ai 2 ...,"[1246042405561872384, 1533187263156432896, 116...","[[covid, gouvernementdelahonte, nouvelordremon...","[Paris, Paris, Paris, Paris, Paris, Paris, Par...",0 False 1 False 2 False 3 Fals...
2,1466835011454189571,0 RightAndFree_ 5 RightAndFree_ 6 ...,"[1219232377605644289, 172377413, 1219232377605...","[[ukraine, russia], [ny19], [russia, ukraine],...","[nan, nan, nan, nan]",0 False 5 False 6 False 16 Fals...
3,4813333392,0 PansareKetan 8 PansareKetan 13 Pa...,"[81386285, 576533046, 849006404, 19897138]","[[shivsena, covid], [bjpjawabdo], [sanjaypande...","[nan, nan, nan, nan]",0 False 8 False 13 False 19 Fals...
4,2471461255,0 CanadaRomania 2 CanadaRomania 4 ...,"[121514648, 157981564, 14353202, 14353202, 410...","[[russia, ukraine, indonesia], [foodsecurity],...","[Bucharest, Romania, Bucharest, Romania, Bucha...",0 True 2 True 4 True 5 True 6 ...
...,...,...,...,...,...,...
72,47941897,0 ma000111 1 ma000111 2 ma000111 3...,"[47941897, 1167016822027112449, 47941897, 4794...","[[zelenskyy, warcrimes, donetsk, ukraine, ukra...","[England , England , England , England , Engla...",0 False 1 False 2 False 3 Fals...
73,2865524457,0 michelangela_K 2 michelangela_K 5 ...,"[23372297, 1364378809, 1091395730307190785, 33...","[[ukraine], [affenpocken, rki], [shinzoabe], [...","[Buxtehude, Buxtehude, Buxtehude, Buxtehude, B...",0 False 2 False 5 False 13 Fals...
74,1056303825995337730,1 MesarutF 4 MesarutF 12 MesarutF 1...,"[1496161188077621251, 1505481949854502915, 100...","[[donbas, ukraine, ukrainewarnews, stoprussia]...","[nan, nan, nan, nan, nan]",1 False 4 False 12 False 13 Fals...
75,3245630026,0 VerachikaV 2 VerachikaV 3 Verach...,"[191145819, 23372297, 1231514832479948802, 206...","[[forgottensoulshour, rescuedog, rehomehour, r...","[nan, nan, nan, nan, nan, nan, nan, nan, nan]",0 False 2 False 3 False 4 Fals...


In [29]:
#convert nested lists on set
outer_list = [] 
def lists_to_list(nested_lists): 
    for el in nested_lists: 
        if type(el) == list: 
            lists_to_list(el) 
        else: 
            outer_list.append(el)
    return set(outer_list)

In [30]:
outer_list = [] 
df_nodes['hashtag']=df_nodes['hashtag'].map(lists_to_list)
outer_list = [] 
df_nodes['retweet']=df_nodes['retweet'].map(lists_to_list)
outer_list = [] 
df_nodes['name']=df_nodes['name'].map(lists_to_list)
outer_list = [] 
df_nodes['location']=df_nodes['location'].map(lists_to_list)
outer_list = [] 
df_nodes['verified']=df_nodes['verified'].map(lists_to_list)

In [45]:
boolean={'True','False'}
for b in df_nodes['verified']:
    print(b.intersection(boolean)) #=={'True','False'}:
       # b=='True'

set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()


In [46]:
df_nodes

,user_id,name,retweet,hashtag,location,verified
0,1357718765941710848,{PeterJohnKing4},{1219232377605644289},"{ukraine, russia}",{Somewhere},{False}
1,1151202637582348294,"{PeterJohnKing4, hephaistos_ai}","{1533187263156432896, 24343635, 15422873631492...","{plandemic, staycovidsafe, vote, vacci, republ...","{Paris, Somewhere}",{False}
2,1466835011454189571,"{PeterJohnKing4, RightAndFree_, hephaistos_ai}","{1533187263156432896, 24343635, 15422873631492...","{plandemic, staycovidsafe, vote, trumpisatrait...","{Paris, nan, Somewhere, nan, nan, nan}",{False}
3,4813333392,"{PeterJohnKing4, RightAndFree_, PansareKetan, ...","{1533187263156432896, 24343635, 15422873631492...","{plandemic, staycovidsafe, vote, trumpisatrait...","{Paris, nan, nan, nan, Somewhere, nan, nan, na...",{False}
4,2471461255,"{PeterJohnKing4, CanadaRomania, RightAndFree_,...","{1533187263156432896, 33285177, 24343635, 4105...","{trumpisatraitor, republican, covidvaccine, tr...","{Bucharest, Romania, Paris, nan, nan, nan, Som...","{False, True}"
...,...,...,...,...,...,...
72,47941897,"{PeterJohnKing4, Douglas17569801, CanadaRomani...","{791612573531668481, 2932618967, 1342474419944...","{, buckwheat, nutrition, trumpisatraitor, majo...","{nan, DCEU Earth 666, MCU Earth 616, nan, Repu...","{False, True}"
73,2865524457,"{PeterJohnKing4, Douglas17569801, CanadaRomani...","{791612573531668481, 2932618967, 3315313691, 1...","{, buckwheat, nutrition, trumpisatraitor, majo...","{nan, DCEU Earth 666, MCU Earth 616, nan, Repu...","{False, True}"
74,1056303825995337730,"{PeterJohnKing4, Douglas17569801, CanadaRomani...","{791612573531668481, 2932618967, 3315313691, 1...","{, buckwheat, nutrition, trumpisatraitor, majo...","{nan, DCEU Earth 666, MCU Earth 616, nan, Repu...","{False, True}"
75,3245630026,"{PeterJohnKing4, Douglas17569801, CanadaRomani...","{791612573531668481, 2932618967, 3315313691, 1...","{, buckwheat, nutrition, trumpisatraitor, majo...","{nan, DCEU Earth 666, MCU Earth 616, nan, nan,...","{False, True}"


In [37]:
proukr=set(["slavaukraini"]) #1 cat
prorus=set(["istandwithputin", "istandwithrussia"]) #2 cat
pax=set(["stopwarinukraine"]) #3 cat
cat=set.union(proukr,prorus,pax)
count1=0
#count2=0
#count3=0
count4=0
index1=[]
#index2=[]
#index3=[]
index4=[]

for i, row in enumerate(df_IDK1['hashtag']): 
    if row.intersection(cat)==set():#no intersezione   
        #print(row.intersectionn(cat))
        #print("l'utente {}  appartiene alla 4 categoria".format(row.intersection(cat)))
        #print("4 categoria", row)
        count4 += 1
        index4.append(i)
        
    else: #c'è intersezione: uno della 3 categorie
        #print("non è la 4 categoria perchè contiene",row.intersection(cat))
       # if row.intersection(proukr)!=set(): #c'è intersez con proukr
           # count1 += 1
            index1.append(i) #aggiungo alle rige da tgliere
            print("l'utente {} non  appartiene alla 4 categoria perchè usa {}".format(i,row.intersection(cat)))
       # elif row.intersection(prorus)!=set():
          #  count2 += 1
           # index2.append(i)
        #elif row.intersection(pax)!=set():
           # count3 += 1
           #index3.append(i)

l'utente 28 non  appartiene alla 4 categoria perchè usa {'slavaukraini'}
l'utente 29 non  appartiene alla 4 categoria perchè usa {'slavaukraini'}
l'utente 30 non  appartiene alla 4 categoria perchè usa {'slavaukraini'}
l'utente 31 non  appartiene alla 4 categoria perchè usa {'slavaukraini'}
l'utente 32 non  appartiene alla 4 categoria perchè usa {'slavaukraini'}
l'utente 33 non  appartiene alla 4 categoria perchè usa {'slavaukraini'}
l'utente 34 non  appartiene alla 4 categoria perchè usa {'slavaukraini'}
l'utente 35 non  appartiene alla 4 categoria perchè usa {'slavaukraini'}
l'utente 36 non  appartiene alla 4 categoria perchè usa {'slavaukraini'}
l'utente 37 non  appartiene alla 4 categoria perchè usa {'slavaukraini'}
l'utente 38 non  appartiene alla 4 categoria perchè usa {'slavaukraini'}
l'utente 39 non  appartiene alla 4 categoria perchè usa {'slavaukraini'}
l'utente 40 non  appartiene alla 4 categoria perchè usa {'slavaukraini'}
l'utente 41 non  appartiene alla 4 categoria perchè

In [29]:
print('proukr = {} \n'
     'prourus = {} \n'
     'pax = {} \n'
     'nocare = {} '.format(count1,count2,count3,count4))

proukr = 391 
prourus = 0 
pax = 0 
nocare = 28 


## 3 parte categorizzazione tweet e utenti di proukr, prorus, pax

## 4 parte: edge tra utenti nel dataset

In [67]:
df_IDK1['retweet']

0     {1498176452084256772, 1219232377605644289, 135...
1     {1359617172033724416, 14106476, 14981764520842...
2     {1521787270260719616, 923231030160400384, 1359...
3     {1521787270260719616, 923231030160400384, 1359...
4     {1521787270260719616, 923231030160400384, 1359...
                            ...                        
73    {720139699, 14106476, 1458038317619073044, 316...
74    {720139699, 14106476, 1458038317619073044, 316...
75    {720139699, 14106476, 1458038317619073044, 316...
76    {720139699, 14106476, 1458038317619073044, 316...
77    {720139699, 14106476, 1458038317619073044, 316...
Name: retweet, Length: 78, dtype: object